# Cryptocurrencies, stock markets and COVID-19

The discussion about up to what level do investors look into the real economy to make decisions is a much debated one. The positive and behavioral approaches -along with all their intersections- are research rabbit holes that seek to shed some light on the logic behind investment allocation decisions. However, no matter how far into one side or another one was standing, Covid-19 paralyzed the world. Therefore, it was discounted that the largest economic shock of our generation would be reflected in the forever record breaking stock market. But what about the crypto market? After the Bitcoin halving that took place in May 2020, a bull market was expected in the crypto universe. If history serves as a predictor of the future -another controversial debate-, 2021 was expected to be a reprise of the 2013 and 2017 bullish trends. 

With more than 18 months into the pandemic, we know that the stock market took a hit, followed by a fast recovery that led to new all time highs. On the other hand, we saw that Covid was not anough to cancel the crypto-party expected for this year.

Let's see how all these facts look like graphically.

In [1]:
#%%capture
# Install some necessary packages
#!pip install functools
#!pip install --upgrade covid
#!pip install fastquant
#!pip install yfinance

In [2]:
# Import the required libraries and set some parameters
import numpy as np
import pandas as pd
import fastquant as fq
from datetime import datetime as dt
import yfinance as yf
from functools import reduce
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20, 8)
%config InlineBackend.figure_format='retina'

In [3]:
# Import COVID data from Our World in Data
Covid_data = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv')

In [4]:
# Check the dataset
Covid_data.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN


In [5]:
# Keep cuntry ID, date, daily number of cases and daily number of deaths
Covid_data = Covid_data[['iso_code','date','new_cases','new_deaths']]

In [6]:
# Check the new dataset
Covid_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125965 entries, 0 to 125964
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   iso_code    125965 non-null  object 
 1   date        125965 non-null  object 
 2   new_cases   119150 non-null  float64
 3   new_deaths  108371 non-null  float64
dtypes: float64(2), object(2)
memory usage: 3.8+ MB


In [ ]:
# See the list of countries
Covid_data.iso_code.unique()

In [ ]:
# Generate a list with the 38 OECD countries
OECD_list = ['AUS','AUT','BEL','CAN','CHL','COL','CRI','CZE','DNK','ESP','EST','FIN','FRA','GBR','DEU','GRC','HUN','IRL','ISL',
            'ISR', 'ITA','JPN','KOR','LVA','LTU','LUX','MEX','NLD','NOR', 'POL', 'NZL','PRT','SVK','SVN','SWE','CHE','TUR','USA']

In [ ]:
# Make a mask to keep only OECD countries
mask_OECD = Covid_data['iso_code'].apply(lambda x: any(country in x for country in OECD_list))

In [ ]:
# Apply the mask and check that we have only kept the desired countries
Covid_OECD = Covid_data[mask_OECD]
Covid_OECD['iso_code'].unique()

In [ ]:
# Generate the OECD sum of cases
Covid_OECD_final = Covid_OECD.groupby('date').agg({'new_cases':'sum', 'new_deaths': 'sum'}).reset_index()
Covid_OECD_final.info()

In [ ]:
# Check descriptive statistics
Covid_OECD_final.describe()

In [ ]:
# Found a negative value that doesn't make sense. I check which date it corresponds to. 
Covid_OECD_final[Covid_OECD_final.new_deaths<0]

The negative death count corresponds to May 25th 2020. I will look for further information on the OECD dataset.

In [ ]:
Covid_OECD[Covid_OECD.new_deaths<0]

We can see that different days for differenc countries have a negative value for "confirmed new deaths". This is consistent with the interactive online graph from Our World in data: https://ourworldindata.org/explorers/coronavirus-data-explorer?zoomToSelection=true&time=2020-03-01..latest&facet=none&pickerSort=asc&pickerMetric=location&Metric=Confirmed+deaths&Interval=New+per+day&Relative+to+Population=false&Align+outbreaks=false&country=USA~GBR~CAN~DEU~ITA~IND

It is the case that many countries modified or recheck thaeir facts with respect to previous days reports. Thefefore, the negative values correspond to rectifications of previous reports.

In [ ]:
# Create two new columns with moving averages
Covid_OECD_final['cases_rolling'] = Covid_OECD_final['new_cases'].rolling(window=7).mean()
Covid_OECD_final['deaths_rolling'] = Covid_OECD_final['new_deaths'].rolling(window=7).mean()
Covid_OECD_final.head(n=10)

In [ ]:
# Set date as index
Covid_OECD_final['date'] = pd.to_datetime(Covid_OECD_final['date']) 
Covid_OECD_final.set_index('date', inplace=True)
Covid_OECD_final.info()

## Stock data

Since we are using Covid data for OECD countries, we are going to compare these numbers with 5 of its biggest economies: US 🇺🇸 (S&P500), Germany 🇩🇪 (DAX), Great Britain 🇬🇧(FTSE 100), Japan 🇯🇵 (Nikkei 225), and Canada 🇨🇦 (S&P/TSX) 

In [ ]:
tickers = ["^GSPC", "^GDAXI", "^FTSE", "^N225", "^GSPTSE"]
from_date = "2020-01-01"
to_date = dt.today()
stocks_data = yf.download(" ".join(tickers), start=from_date, end=to_date)
stocks_data.head()

In [ ]:
stock_markets = stocks_data['Adj Close']
for tick in tickers:
    stock_markets[tick + '_rolling'] = stock_markets[tick].rolling(window=7,min_periods=4).mean()
stock_markets.head(n=10)

In [ ]:
markets = ['S&P500_US','DAX_GER','LSE_GBR','NIKKEI_JPN','TSX_CAN']
stock_markets = stock_markets.iloc[: , 5:]
stock_markets.columns = markets
stock_markets.head(n=10)

## Crypto data

In [ ]:
#set tomeframe
from_date = "2019-12-31"
to_date = dt.today().strftime('%Y-%m-%d')

In [ ]:
#pull the data
BTC = fq.get_crypto_data("BTC/USDT", from_date, str(to_date))
print('1/5 done')
ETH = fq.get_crypto_data("ETH/USDT", from_date, str(to_date)) 
print('2/5 done')
BNB = fq.get_crypto_data("BNB/USDT", from_date, str(to_date)) 
print('3/5 done')
ADA = fq.get_crypto_data("ADA/USDT", from_date, str(to_date)) 
print('4/5 done')
XRP = fq.get_crypto_data("XRP/USDT", from_date, str(to_date)) 
print('DONE')

In [ ]:
# Keep close price and set time as index
BTC = BTC[['close']].rename(columns={"close": "BTC_close"})
BTC.index.names = ['Date']
ETH = ETH[['close']].rename(columns={"close": "ETH_close"})
ETH.index.names = ['Date']
BNB = BNB[['close']].rename(columns={"close": "BNB_close"})
BNB.index.names = ['Date']
ADA = ADA[['close']].rename(columns={"close": "ADA_close"})
ADA.index.names = ['Date']
XRP = XRP[['close']].rename(columns={"close": "XRP_close"})
XRP.index.names = ['Date']

In [ ]:
BTC.head()

In [ ]:
# Merge all datasets
Coins = [BTC,ETH,BNB,ADA,XRP]
Cryptos = reduce(lambda left,right: pd.merge(left,right,left_index=True, right_index=True,
                                            how='outer'), Coins)
Cryptos.head()

In [ ]:
# Check from and to dates
print(f"start date: {Cryptos.index.min()} ")
print(f"end date: {Cryptos.index.max()}")

In [ ]:
# Check that the merge was correct
Cryptos.info()

In [ ]:
# Create 7 days MA for each coin
coins = ['BTC_close','ETH_close','BNB_close','ADA_close','XRP_close']
for coin in coins:
    Cryptos[coin + '_rolling'] = Cryptos[coin].rolling(window=7).mean()
Cryptos.head(n=10)

## Comparing the crypto and COVID evolution

In [ ]:

Coins = ['BTC_close_rolling','ETH_close_rolling','BNB_close_rolling','ADA_close_rolling','XRP_close_rolling']
colors = ['darkorange','mediumblue','darkgoldenrod','royalblue','mediumturquoise']
ncolor = 0
#plt.rcParams["figure.facecolor"] = "w"
for coin in Coins:
    # Normalize values
    Cryptos[coin] = (Cryptos[coin] - Cryptos[coin].min()) / (Cryptos[coin].max() - Cryptos[coin].min())
    ax = Cryptos[coin].plot(color=colors[ncolor])
    ax.legend()
    ncolor+=1
ax.set(facecolor = "lightgrey")
ax1 = ax.twinx()
Covid_OECD_final['cases_rolling'].plot(c = 'red', ax = ax1, linewidth=3)
ax1.legend(loc='lower right')

plt.suptitle('Top 5 cryptocurrencies and Covid-19 cases evolution', fontsize=18)
ax.set_xlabel('Date', fontsize=14)
ax.set_ylabel('Normalized cryptocurrency prices', fontsize=14)
ax1.set_ylabel('Nominal Covid-19 cases', fontsize=14)
plt.grid(c='w')
plt.show()

In [ ]:
ncolor = 0
for coin in Coins:
    # Normalize values
    Cryptos[coin] = (Cryptos[coin] - Cryptos[coin].min()) / (Cryptos[coin].max() - Cryptos[coin].min())
    ax = Cryptos[coin].plot(color=colors[ncolor])
    ax.legend()
    ncolor+=1
ax1 = ax.twinx()
ax.set(facecolor = "lightgrey")
Covid_OECD_final['deaths_rolling'].plot(c = 'black', ax = ax1, linewidth=3)
ax1.legend(loc='lower right')
plt.suptitle('Top 5 cryptocurrencies and Covid-19 deaths evolution', fontsize=18)
ax.set_xlabel('Date', fontsize=14)
ax.set_ylabel('Normalized cryptocurrency prices', fontsize=14)
ax1.set_ylabel('Nominal Covid-19 deaths', fontsize=14)
plt.grid(c='w')
plt.show()

## Comparing the stocks and Covid evolution

In [ ]:
tickers = ['S&P500 🇺🇸','DAX_GER','LSE_GBR','NIKKEI_JPN','TSX_CAN']
colors = ['green','tomato','mediumvioletred','cornflowerblue','dimgray']
ncolor = 0
#plt.rcParams["figure.facecolor"] = "w"
for tick in tickers:
    # Normalize values
    stock_markets[tick] = (stock_markets[tick] - stock_markets[tick].min()) / (stock_markets[tick].max() - stock_markets[tick].min())
    ax = stock_markets[tick].plot(color=colors[ncolor])
    ax.legend()
    ncolor+=1
ax.set(facecolor = "lightgrey")
ax1 = ax.twinx()
Covid_OECD_final['cases_rolling'].plot(c = 'red', ax = ax1, linewidth=3)
ax1.legend(loc='lower right')

plt.suptitle('5 representative developed stock markets and Covid-19 cases evolution 🇨🇦', fontsize=18)
ax.set_xlabel('Date', fontsize=14)
ax.set_ylabel('Normalized stock prices', fontsize=14)
ax1.set_ylabel('Nominal Covid-19 cases', fontsize=14)
plt.grid(c='w')
plt.show()

In [ ]:
colors = ['green','tomato','mediumvioletred','cornflowerblue','dimgray']
ncolor = 0
#plt.rcParams["figure.facecolor"] = "w"
for tick in tickers:
    # Normalize values
    stocks_data[tick] = (stocks_data[tick] - stocks_data[tick].min()) / (stocks_data[tick].max() - stocks_data[tick].min())
    ax = stocks_data[tick].plot(color=colors[ncolor])
    ax.legend()
    ncolor+=1
ax.set(facecolor = "lightgrey")
ax1 = ax.twinx()
ax.set(facecolor = "lightgrey")
Covid_OECD_final['deaths_rolling'].plot(c = 'black', ax = ax1, linewidth=3)
ax1.legend(loc='lower right')
plt.suptitle('5 representative developed stock markets and Covid-19 deaths evolution', fontsize=18)
ax.set_xlabel('Date', fontsize=14)
ax.set_ylabel('Normalized index prices', fontsize=14)
ax1.set_ylabel('Nominal Covid-19 deaths', fontsize=14)
plt.grid(c='w')
plt.show()